In [32]:
import numpy as np

from numba import jit
from numba import njit, prange, set_num_threads
from numba import cuda

In [1]:
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("hello_world.pyx")
)

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [2]:
import os
import numpy as np

In [2]:
os.environ["C_INCLUDE_PATH"] = np.get_include()

In [3]:
import pyximport

In [4]:
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x7f01f876cc10>)

In [3]:
from rocauc_pairwise.utils import get_non_unique_labels_count, get_non_unique_borders, get_inverse_argsort

In [1]:
from rocauc_pairwise.utils_old import get_non_unique_labels_count, get_non_unique_borders, get_inverse_argsort

In [5]:
%%timeit 
_, _ = get_non_unique_labels_count(y_true, 
                                   y_pred, 
                                   np.argsort(y_pred))

491 µs ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
y_true = [0, 1, 0, 1, 0, 1, 1, 0, 1, 0]
y_pred = [0.5, 0.5, 0.5, 0.1, 0.05, 0.02, 0.02, 0.02, 0.02, 0.02]
y_pred = np.array(y_pred)
y_true = np.array(y_true)

In [4]:
y_true = np.random.randint(0, 2, 10_000)
y_pred = np.random.randn(10_000)

In [6]:
def test_get_inverse_argsort():
    inversed_argsort = get_inverse_argsort(y_true, y_pred)
    sorted_array = []
    for i in inversed_argsort:
        sorted_array.append(y_pred[i])
    sorted_array_numpy = np.sort(y_pred)[::-1]
    print(sorted_array)
    print(sorted_array_numpy)
    
    assert (np.array(sorted_array) == sorted_array_numpy).all()

In [7]:
test_get_inverse_argsort()

[0.05, 0.1, 0.5, 0.5, 0.5, 0.02, 0.02, 0.02, 0.02, 0.02]
[4 3 2 1 0 5 6 9 8 7]


NameError: name 'sorted_array_numpy' is not defined

In [45]:
@jit(parallel=False, nopython=True)
def sigmoid_pairwise_cpu(exp_pred, y_true):
    eps = 1e-20
    log_loss_ = 0.
    size = len(exp_pred)
    for i in prange(size):
        k = (13 * i) % size
        for j in range(k + 1):
            P_hat = 0.5 *(y_true[i] - y_true[j]) + 0.5
            P = 1.0 / (1.0 + (exp_pred[j] / exp_pred[i]))
            cur_log_loss = P_hat*np.log(P + eps) + (1.0 - P_hat)*np.log(1 - P + eps)
            if i == j:
                cur_log_loss *= 0.5
            log_loss_ += cur_log_loss / size**2
    return log_loss_

In [37]:
9929 * 1 % 10

9

In [37]:
N = 120
y_true = np.random.randint(0, 2, N).astype(np.float32)
y_pred = np.random.randn(N).astype(np.float32)

In [13]:
%%time
sigmoid_pairwise(np.exp(y_true), y_true)

CPU times: user 28.4 s, sys: 35 µs, total: 28.4 s
Wall time: 28.4 s


-0.2516051580143744

In [17]:
%%time
sigmoid_pairwise(np.exp(y_true), y_true)

CPU times: user 1min 21s, sys: 20 ms, total: 1min 21s
Wall time: 11.8 s


-0.2516028803233659

In [39]:
%%time
sigmoid_pairwise(np.exp(y_true), y_true)

CPU times: user 3min 11s, sys: 43.8 ms, total: 3min 12s
Wall time: 17.4 s


-0.25160318854809316

In [42]:
%%time
sigmoid_pairwise(np.exp(y_true), y_true)

CPU times: user 3min 11s, sys: 35.9 ms, total: 3min 11s
Wall time: 17 s


-0.25160318854809316

In [38]:
@cuda.jit 
def sigmoid_pairwise(exp_pred, y_true, output, size):
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x

    eps = 1e-20
    i = (bid*bdim) + tid

    if i < size:
        log_loss_ = 0.
        for j in range(i + 1):
            P_hat = 0.5 *(y_true[i] - y_true[j]) + 0.5
            P = 1.0 / (1.0 + (exp_pred[j] / exp_pred[i]))
            cur_log_loss = P_hat*math.log(P + eps) + (1.0 - P_hat)*math.log(1 - P + eps)
            if i == j:
                cur_log_loss *= 0.5
            log_loss_ += cur_log_loss / size**2
        cuda.atomic.add(output, 0, log_loss_)

In [4]:
import math

In [35]:
N_THREADS = 512

In [39]:
output = np.zeros(1, dtype=np.float32)

In [40]:
output

array([0.], dtype=float32)

In [51]:
N_THREADS = 10

In [52]:
sigmoid_pairwise[N_THREADS, math.ceil(y_true.shape[0] / N_THREADS)](np.exp(y_pred), y_true, output, y_true.shape[0])


/home/dmitry/anaconda3/envs/rocauc_pairwise/lib/python3.8/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 10 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [53]:
output

array([-1.428094], dtype=float32)

In [54]:
set_num_threads(6)
sigmoid_pairwise_cpu(np.exp(y_pred), y_true)

-0.4756685186661678